## 编写SimpleWriteCode动作

In [3]:
import re
import asyncio
from metagpt.actions import Action

class SimpleWriteCode(Action):

    PROMPT_TEMPLATE = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    def __init__(self, name="SimpleWriteCode", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, instruction: str):

        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r'```python(.*)```'
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

## 设计SimpleCoder角色

In [7]:
import re
import asyncio
from metagpt.actions import Action
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

class SimpleWriteCode(Action):

    PROMPT_TEMPLATE = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    def __init__(self, name="SimpleWriteCode", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, instruction: str):

        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r'```python(.*)```'
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

class SimpleCoder(Role):
    def __init__(
        self,
        name: str = "Alice",
        profile: str = "SimpleCoder",
        **kwargs,
    ):
        super().__init__(name, profile, **kwargs)
        self._init_actions([SimpleWriteCode])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self._rc.todo}")
        todo = self._rc.todo  # todo will be SimpleWriteCode()

        msg = self.get_memories(k=1)[0]  # find the most recent messages

        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile,
                      cause_by=type(todo))

        return msg




In [9]:
async def main():
    msg = "write a function that calculates the sum of a list"
    role = SimpleCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

# asyncio.run(main())
await main()

2024-01-12 15:31:16.401 | INFO     | __main__:main:4 - write a function that calculates the sum of a list
2024-01-12 15:31:16.407 | INFO     | __main__:_act:47 - Alice(SimpleCoder): ready to SimpleWriteCode


```python
def sum_list(lst):
    return sum(lst)

# Test case 1
print(sum_list([1, 2, 3, 4, 5]))  # Expected output: 15

# Test case 2
print(sum_list([-1, 0, 1]))  # Expected output: 0
```

2024-01-12 15:31:20.843 | INFO     | __main__:main:6 - SimpleCoder: 
def sum_list(lst):
    return sum(lst)

# Test case 1
print(sum_list([1, 2, 3, 4, 5]))  # Expected output: 15

# Test case 2
print(sum_list([-1, 0, 1]))  # Expected output: 0



## 多动作Agent  

In [13]:
import os
import re
import subprocess
import asyncio

import fire
import sys
from metagpt.llm import LLM
from metagpt.actions import Action
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

### 编写 SimpleRunCode 动作

In [10]:
class SimpleRunCode(Action):
    def __init__(self, name="SimpleRunCode", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, code_text: str):
        # 在Windows环境下，result可能无法正确返回生成结果，在windows中在终端中输入python3可能会导致打开微软商店
        result = subprocess.run(["python3", "-c", code_text], capture_output=True, text=True)
        # 采用下面的可选代码来替换上面的代码
        # result = subprocess.run(["python", "-c", code_text], capture_output=True, text=True)
        # import sys
        # result = subprocess.run([sys.executable, "-c", code_text], capture_output=True, text=True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

### 定义 RunnableCoder 角色

In [11]:
class RunnableCoder(Role):
    def __init__(
        self,
        name: str = "Alice",
        profile: str = "RunnableCoder",
        **kwargs,
    ):
        super().__init__(name, profile, **kwargs)
        self._init_actions([SimpleWriteCode, SimpleRunCode])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: 准备 {self._rc.todo}")
        # 通过在底层按顺序选择动作
        # todo 首先是 SimpleWriteCode() 然后是 SimpleRunCode()
        todo = self._rc.todo

        msg = self.get_memories(k=1)[0] # find the most recent messages
        result = await todo.run(msg.content)

        msg = Message(content=result, role=self.profile, cause_by=type(todo))
        self._rc.memory.add(msg)
        return msg

### 运行 RunnableCoder 角色

In [14]:
import asyncio

async def main():
    msg = "write a function that calculates the sum of a list"
    role = RunnableCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

# asyncio.run(main())
await main()

2024-01-12 15:44:57.758 | INFO     | __main__:main:6 - write a function that calculates the sum of a list
2024-01-12 15:44:57.769 | INFO     | __main__:_act:13 - Alice(RunnableCoder): 准备 SimpleWriteCode


```python
def sum_list(lst):
    return sum(lst)

# Test cases
print(sum_list([1, 2, 3, 4, 5]))  # Expected output: 15
print(sum_list([-1, 0, 1]))  # Expected

2024-01-12 15:45:02.763 | INFO     | __main__:_act:13 - Alice(RunnableCoder): 准备 SimpleRunCode
2024-01-12 15:45:02.846 | INFO     | __main__:run:13 - code_result='15\n0\n'
2024-01-12 15:45:02.848 | INFO     | __main__:main:8 - RunnableCoder: 15
0



 output: 0
```
